In [1]:
from pyscf import scf,gto
import numpy as np
import inspect
from FcMole import FcM
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf  #### very important
from pyscf.hessian import rhf as hrhf # without those two mf.Gradients() and mf.Hessian() don't work
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

First compare gradient analytical from fcm with the one via finite differences on geometry


In [2]:
mol1=FcM(fcs=[.001,-.001],atom="C 0 0 0; O 0 0 1.8",unit="Bohr",basis="STO-3G")

In [3]:
mf1=scf.RHF(mol1)
mf1.scf(dm0=mf1.init_guess_by_atom())

converged SCF energy = -111.05722603438


-111.05722603437978

In [4]:
mol2=FcM(fcs=[.001,-.001],atom="C 0 0 0; O 0 0 1.81",unit="Bohr",basis="STO-3G")
mol3=FcM(fcs=[.001,-.001],atom="C 0 0 0; O 0 0 1.79",unit="Bohr",basis="STO-3G")
mf2=scf.RHF(mol2)
mf2.scf(dm0=mf2.init_guess_by_atom())
mf3=scf.RHF(mol3)
mf3.scf(dm0=mf3.init_guess_by_atom())

converged SCF energy = -111.067847863603
converged SCF energy = -111.046103208766


-111.04610320876637

In [5]:
fdg=(mf2.energy_tot()-mf3.energy_tot())/.02

In [6]:
fdg

-1.0872327418411487

In [7]:
g1=mf1.Gradients()
g1.run()

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000     1.0869705410
1 O    -0.0000000000    -0.0000000000    -1.0869705410
----------------------------------------------


In [8]:
fdh=(mf2.energy_tot()-2*mf1.energy_tot()+mf3.energy_tot())/.01**2
fdh

5.009963900022285

In [9]:
hs1=mf1.Hessian()
hs1.kernel()

/home/giorgiod/miniconda3/lib/python3.7/site-packages/pyscf/lib/chkfile.py:71: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return val.value


array([[[[-6.03872521e-01, -9.63563135e-17,  4.11123616e-16],
         [-1.00597157e-16, -6.03872521e-01,  6.81166538e-16],
         [ 4.11123615e-16,  6.81166537e-16,  5.00953800e+00]],

        [[ 6.03872521e-01,  1.00597157e-16, -4.11123615e-16],
         [ 9.28868665e-17,  6.03872521e-01, -6.81166537e-16],
         [-4.11123616e-16, -6.81166538e-16, -5.00953800e+00]]],


       [[[ 6.03872521e-01,  9.28868665e-17, -4.11123616e-16],
         [ 1.00597157e-16,  6.03872521e-01, -6.81166538e-16],
         [-4.11123615e-16, -6.81166537e-16, -5.00953800e+00]],

        [[-6.03872521e-01, -6.19986447e-17,  4.11123616e-16],
         [-4.54314803e-17, -6.03872521e-01,  6.81166538e-16],
         [ 4.11123615e-16,  6.81166537e-16,  5.00953800e+00]]]])

In [10]:
mol0=gto.M(atom="C 0 0 0; O 0 0 1.8",unit="Bohr",basis="STO-3G")
mf0=scf.RHF(mol0)
mf0.scf()

g0=mf0.Gradients()
g0.kernel()
hs0=mf0.Hessian()
hs0.kernel() 

converged SCF energy = -111.064463936466
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000     1.0871732099
1 O    -0.0000000000    -0.0000000000    -1.0871732099
----------------------------------------------


array([[[[-6.03985104e-01,  5.82015360e-16,  1.84611257e-16],
         [ 6.13117995e-16, -6.03985104e-01, -3.85299017e-16],
         [ 1.84611256e-16, -3.85299019e-16,  5.00937821e+00]],

        [[ 6.03985104e-01, -6.13117995e-16, -1.84611256e-16],
         [-5.82015360e-16,  6.03985104e-01,  3.85299019e-16],
         [-1.84611257e-16,  3.85299017e-16, -5.00937821e+00]]],


       [[[ 6.03985104e-01, -5.82015360e-16, -1.84611257e-16],
         [-6.13117995e-16,  6.03985104e-01,  3.85299017e-16],
         [-1.84611256e-16,  3.85299019e-16, -5.00937821e+00]],

        [[-6.03985104e-01,  5.73066100e-16,  1.84611257e-16],
         [ 5.04857268e-16, -6.03985104e-01, -3.85299017e-16],
         [ 1.84611256e-16, -3.85299019e-16,  5.00937821e+00]]]])

In [11]:
from alch_deriv import alch_deriv

The formula for the gradient is stated in Pople's article (Eq.21) as: 
$$ \frac{\partial E}{\partial x}= \sum_{\mu\nu}P_{\mu\nu}\frac{\partial H_{\mu\nu}}{\partial x}+\frac{1}{2}\sum_{\mu\nu\lambda\sigma}
P_{\mu\nu}P_{\lambda\sigma}\frac{\partial}{\partial x}(\mu \lambda | | \nu\sigma)+\frac{\partial V_{nuc}}{\partial x} 
-\sum_{\mu\nu}W_{\mu\nu}\frac{\partial S_{\mu\nu}}{\partial x}
$$
$W$ is an energy weighted density matrix:
$$ W_{\mu\nu}= \sum_i ^{mo.occ.} \epsilon_i c_{\mu i} c_{\nu i}^\dagger
$$

In [12]:
(U,dP)=alch_deriv(mf0)
P=mf0.make_rdm1()
P1=mf1.make_rdm1()

In [13]:
np.linalg.norm(dP-(P1-P))

6.705322478643819e-07

First piece:
$$ \frac{\partial}{\partial Z} ( P \frac{\partial H^{(1)}}{\partial x})= 
\frac{\partial P}{\partial Z}\frac{\partial H^{(1)}}{\partial x}+ P \frac{\partial^2 H^{(1)}}{\partial x \partial Z}
$$
$\frac{\partial^2 H^{(1)}}{\partial x \partial Z}$ is trivially $\frac{\partial H^{(1)}}{\partial x}$ divided by the atom charge

In [14]:
ga=np.zeros((2,3))
ga[0]+=np.einsum('xij,ij->x', g0.hcore_generator()(0),dP)
ga[1]+=np.einsum('xij,ij->x', g0.hcore_generator()(1),dP)
print(ga)
ga[0]+=np.einsum('xij,ij->x', g1.hcore_generator()(0)-g0.hcore_generator()(0),P)
ga[1]+=np.einsum('xij,ij->x', g1.hcore_generator()(1)-g0.hcore_generator()(1),P)
print(ga)

[[ 4.34158318e-18  5.79445587e-18 -4.42827506e-03]
 [-4.34158318e-18 -5.79445587e-18  4.42827506e-03]]
[[ 4.35688667e-18  5.79657387e-18 -5.24933658e-03]
 [-4.35688667e-18 -5.79657387e-18  5.24933658e-03]]


In [15]:
comp1=np.zeros((2,3))
comp1[0]+=np.einsum('xij,ij->x', g1.hcore_generator()(0),mf1.make_rdm1())
comp1[0]-=np.einsum('xij,ij->x', g0.hcore_generator()(0),mf0.make_rdm1())
comp1[1]+=np.einsum('xij,ij->x', g1.hcore_generator()(1),mf1.make_rdm1())
comp1[1]-=np.einsum('xij,ij->x', g0.hcore_generator()(1),mf0.make_rdm1())
comp1

array([[-3.02497705e-15,  6.55469972e-16, -5.24514437e-03],
       [ 3.02497705e-15, -6.55469972e-16,  5.24514437e-03]])

In [16]:
comp1=np.zeros((2,3,10,10))
comp1[0]=g1.hcore_generator()(0)-g0.hcore_generator()(0)
comp1[1]=g1.hcore_generator()(1)-g0.hcore_generator()(1)

In [17]:
comp2=np.zeros((2,3,10,10))
dL=[.001,-.001]
for atm_id in [0,1]:
    with mol0.with_rinv_at_nucleus(atm_id):
        vrinv = -mol0.intor('int1e_iprinv', comp=3)
    shl0, shl1, p0, p1 = mol0.aoslice_by_atom()[atm_id]
    vrinv*=dL[atm_id]
    vrinv[:,p0:p1] += (g1.get_hcore()-g0.get_hcore())[:,p0:p1]  #bearbeiten
    vrinv += vrinv.transpose(0,2,1)
    comp2[atm_id]=vrinv

In [18]:
np.allclose(comp1,comp2)

True

g0.get_hcore() is the integral $<\chi_\mu |\nabla_r \hat{H}^{(1)}|\chi_\nu> $ is composed by two parts: the first refered to the kintic energy operator which is alchemy invariant, the second which has to be computed is refered to the nuclear electron attraction. <br>
To compute this we use moleintor.getints() using as arguments a mol environment (mol._env) with the added fractional charges and a mol._atm desription that show fractional charges.
Not forget to put a minus sign !!!! 

In [19]:
NUC_FRAC_CHARGE=gto.mole.NUC_FRAC_CHARGE
NUC_MOD_OF=gto.mole.NUC_MOD_OF
PTR_FRAC_CHARGE=gto.mole.PTR_FRAC_CHARGE
denv=mol0._env.copy()
datm=mol0._atm.copy()
fcs=[.001,-.001]
datm[:,NUC_MOD_OF] = NUC_FRAC_CHARGE
for i in range (mol0.natm):
    denv[datm[i,PTR_FRAC_CHARGE]]=fcs[i] 
dH1=-gto.moleintor.getints('int1e_ipnuc_sph',datm,mol0._bas,denv, None,3,0,'s1')   #minus sign !

In [20]:
np.allclose(dH1,g1.get_hcore()-g0.get_hcore())

True

In [21]:
comp2=np.zeros((2,3,10,10))
dL=[.001,-.001]
for atm_id in [0,1]:
    with mol0.with_rinv_at_nucleus(atm_id):
        vrinv = -mol0.intor('int1e_iprinv', comp=3)
    shl0, shl1, p0, p1 = mol0.aoslice_by_atom()[atm_id]
    vrinv*=dL[atm_id]
    vrinv[:,p0:p1] += dH1[:,p0:p1]  #bearbeiten
    vrinv += vrinv.transpose(0,2,1)
    comp2[atm_id]=vrinv
np.allclose(comp1,comp2)

True

In [22]:
fdg=np.zeros((2,3))
fdg[0]+=np.einsum('xij,ij->x', g1.hcore_generator()(0),mf1.make_rdm1())
fdg[0]-=np.einsum('xij,ij->x', g0.hcore_generator()(0),mf0.make_rdm1())
fdg[1]+=np.einsum('xij,ij->x', g1.hcore_generator()(1),mf1.make_rdm1())
fdg[1]-=np.einsum('xij,ij->x', g0.hcore_generator()(1),mf0.make_rdm1())
fdg

array([[-3.02497705e-15,  6.55469972e-16, -5.24514437e-03],
       [ 3.02497705e-15, -6.55469972e-16,  5.24514437e-03]])

In [23]:
ga=np.zeros((2,3))
ga[0]+=np.einsum('xij,ij->x', g0.hcore_generator()(0),dP)
ga[1]+=np.einsum('xij,ij->x', g0.hcore_generator()(1),dP)
#print(ga)
ga[0]+=np.einsum('xij,ij->x', comp2[0],P)
ga[1]+=np.einsum('xij,ij->x', comp2[1],P)
print(ga)

[[ 4.35688667e-18  5.79657387e-18 -5.24933658e-03]
 [-4.35688667e-18 -5.79657387e-18  5.24933658e-03]]


# Second piece:
$$\frac{\partial}{\partial Z} (P_{\mu\nu}P_{\lambda\sigma}\frac{\partial}{\partial x}(\mu \lambda | | \nu\sigma) )$$
here the two electron integral is invariant to alchemy, therefore is sufficient insert the density matrix derivative $dP$ in the following exression. 

In [24]:
#for the ref molecule
aoslices = mol0.aoslice_by_atom()
g2e_part2_0=np.zeros((2,3))
for ia in [0,1]:
    p0, p1 = aoslices [ia,2:]
    vhf = g0.get_veff(mol0, P)
    g2e_part2_0[ia]=(np.einsum('xij,ij->x', vhf[:,p0:p1], P[p0:p1]) * 2)        #   P (Pd/dx(ml||ns))
g2e_part2_0    

array([[-8.06742679e-15, -2.13561485e-14,  1.17925050e+01],
       [ 8.06742679e-15,  2.13561485e-14, -1.17925050e+01]])

In [25]:
aoslices = mol1.aoslice_by_atom()
g2e_part2_1=np.zeros((2,3))
for ia in [0,1]:
    p0, p1 = aoslices [ia,2:]
    vhf = g1.get_veff(mol1, P1)
    g2e_part2_1[ia]=(np.einsum('xij,ij->x', vhf[:,p0:p1], P1[p0:p1]) * 2) 
g2e_part2_1

array([[-4.73738240e-15, -2.22119372e-14,  1.17969086e+01],
       [ 4.73738240e-15,  2.22119372e-14, -1.17969086e+01]])

In [26]:
#check the invariance:
np.allclose(g0.get_veff(mol0, P),g1.get_veff(mol1, P1),atol=1e-4*np.max(g0.get_veff(mol0, P)))

True

In [27]:
aoslices = mol0.aoslice_by_atom()
g2e_part2_d=np.zeros((2,3))
for ia in [0,1]:
    p0, p1 = aoslices [ia,2:]
    vhf = g0.get_veff(mol0, P)
    g2e_part2_d[ia]=(np.einsum('xij,ij->x', vhf[:,p0:p1], dP[p0:p1]) * 2) 
g2e_part2_d

array([[-4.40856729e-18, -5.95308888e-18,  5.26649267e-03],
       [ 3.21049278e-18,  1.39305225e-18, -3.84689150e-03]])

In [28]:
g2e_part2_1-g2e_part2_0

array([[ 3.33004439e-15, -8.55788693e-16,  4.40354970e-03],
       [-3.33004439e-15,  8.55788693e-16, -4.40354970e-03]])

In [29]:
np.allclose(g2e_part2_d,g2e_part2_1-g2e_part2_0,atol=1e-5)

False

# Third piece:
$$-\sum_{\mu\nu}W_{\mu\nu}\frac{\partial S_{\mu\nu}}{\partial x}
$$
Luckily $S$ is invariant in alchemy, therefore the different in gradient is just:$$
-\sum_{\mu\nu}\frac{\partial W_{\mu\nu}}{\partial Z}\frac{\partial S_{\mu\nu}}{\partial x}
$$
### Obtaining derivatives of W
$$W=  \sum_i ^{mo.occ.} \epsilon_i C_{\mu i} C_{\nu i}^\dagger 
$$

$$ \frac{\partial W}{\partial Z_I}= \sum_i ^{mo.occ.} \left( \epsilon_i (CU)_{\mu i} C_{\nu i}^\dagger + 
\epsilon_i C_{\mu i} (CU)^\dagger_{\nu i}   +\frac{\partial \epsilon_i}{\partial Z_I} C_{\mu i} C_{\nu i}^\dagger \right)$$

In [30]:
"""  THE CODE IN g.grad_elec()
dme0 = mf_grad.make_rdm1e(mo_energy, mo_coeff, mo_occ)        W
s1 = mf_grad.get_ovlp(mol)%autocall                         dS/dx
for k, ia in enumerate(atmlst):
    de[k] -= numpy.einsum('xij,ij->x', s1[:,p0:p1], dme0[p0:p1]) * 2        W dS/dx
"""

"  THE CODE IN g.grad_elec()\ndme0 = mf_grad.make_rdm1e(mo_energy, mo_coeff, mo_occ)        W\ns1 = mf_grad.get_ovlp(mol)%autocall                         dS/dx\nfor k, ia in enumerate(atmlst):\n    de[k] -= numpy.einsum('xij,ij->x', s1[:,p0:p1], dme0[p0:p1]) * 2        W dS/dx\n"

In [31]:
#verify that s1 is invariant
s1=g0.get_ovlp(mol0)
np.allclose(g0.get_ovlp(mol0),g1.get_ovlp(mol1))

True

In [32]:
#at first by finite differences 
W1=g1.make_rdm1e()
W0=g0.make_rdm1e()
fd_dW=W1-W0

In [33]:
# W0 can be constructed via C@np.diag(o*e)@C.T
o=mf0.mo_occ
O=np.diag(o)
e=mf0.mo_energy
C=mf0.mo_coeff
S=mf0.get_ovlp()
F=mf0.get_fock()
np.allclose(C@np.diag(e*o)@C.T,W0)

True

In [34]:
#to derive this expression first get dC as 
dC=C@U
#to get d(e) we need to get the fock hamiltonian and than get the new eigenvalues 
g_ijkl=mol0.intor('int2e_sph')
dF2el=np.einsum('ijkl,kl->ij',g_ijkl,dP)-0.5*np.einsum('ijkl,jl->ik',g_ijkl,dP)
dV=DeltaV(mol0,[.001,-.001])

In [35]:
np.allclose(mf1.get_fock()-F,dV+dF2el,atol=1e-5)

True

In [36]:
#try to get e in another way , using Roothans equations FC=SCe
print(np.allclose(np.linalg.inv(S)@F@(C),C@np.diag(e))) # S^-1 F C= C e 
print(np.allclose(np.linalg.inv(S)@F@P,W0))  #S^-1 F (C O C.T) = S^-1 F P = C e O C.T =W !!!!

True
True


In [37]:
e1=np.sort(np.linalg.eig(np.linalg.inv(S)@(F+dV+dF2el))[0])
c1=np.linalg.eig(np.linalg.inv(S)@(F+dV+dF2el))[1]
e1

array([-20.51683893, -11.13600346,  -1.61973963,  -0.72948878,
        -0.68021039,  -0.68021039,  -0.45936705,   0.4005764 ,
         0.4005764 ,   1.38930461])

In [38]:
np.allclose(mf1.mo_energy,e1)

True

In [39]:
np.allclose(mf1.mo_coeff,C+dC)

False

In [40]:
# now we derive C@np.diag(o*e)@C.T  in tree pieces:
#  dC@np.diag(o*e)@C.T+C@np.diag(o*e)@dC.T+C@np.diag(o*(e1-e))@C.T
dW_a=dC@np.diag(o*e)@C.T+C@np.diag(o*e)@dC.T+C@np.diag(o*(e1-e))@C.T
np.max(fd_dW-dW_a)

0.001356425995540206

In [41]:
#or equally
dW_a1=(C+dC)@np.diag(o*(e1))@(C+dC).T-W0
np.max(fd_dW-dW_a1)

0.0013561319839898722

In [42]:
np.max(fd_dW-dW_a1)

0.0013561319839898722

In [43]:
dW_a2=(mf1.mo_coeff)@np.diag(o*(e1))@(mf1.mo_coeff).T-W0
np.max(fd_dW-dW_a2)

4.8249017225998614e-08

In [44]:
#the problem is in the C derivatives
dW_a3=2*C@U@np.diag(o*e)@C.T+2*C@np.diag(o*e)@U.T@C.T+C@np.diag(o*(e1-e))@C.T
np.max(fd_dW-dW_a3)

0.0010752560480206002

In [45]:
dW_a3=np.linalg.inv(S)@(F+dV+dF2el)@(P+dP)-W0
np.max(fd_dW-dW_a3)

7.673113160411305e-07

In [46]:
W1_contr=np.zeros((2,3))  #with fd_dW funziona 
ga_dW_contr=np.zeros((2,3))
W0_contr=np.zeros((2,3))
for ia in [0,1]:
    p0, p1 = mol0.aoslice_by_atom() [ia,2:]
    W1_contr[ia] -= np.einsum('xij,ij->x', s1[:,p0:p1], W1[p0:p1]) * 2
    ga_dW_contr[ia] -= np.einsum('xij,ij->x', s1[:,p0:p1], fd_dW[p0:p1]) * 2
    W0_contr[ia] -= np.einsum('xij,ij->x', s1[:,p0:p1], W0[p0:p1]) * 2
ga_dW_contr,W1_contr-W0_contr

(array([[-5.39803292e-16, -3.51334847e-16,  2.19504694e-05],
        [ 5.39803292e-16,  3.51334847e-16, -2.19504694e-05]]),
 array([[-5.39803292e-16, -3.51334847e-16,  2.19504694e-05],
        [ 5.39803292e-16,  3.51334847e-16, -2.19504694e-05]]))

In [47]:
W1_contr=np.zeros((2,3))
ga_dW_contr=np.zeros((2,3))
W0_contr=np.zeros((2,3))
for ia in [0,1]:
    p0, p1 = mol0.aoslice_by_atom() [ia,2:]
    W1_contr[ia] -= np.einsum('xij,ij->x', s1[:,p0:p1], W1[p0:p1]) * 2
    ga_dW_contr[ia] -= np.einsum('xij,ij->x', s1[:,p0:p1], dW_a3[p0:p1]) * 2
    W0_contr[ia] -= np.einsum('xij,ij->x', s1[:,p0:p1], W0[p0:p1]) * 2
(ga_dW_contr),W1_contr-W0_contr

(array([[ 6.02691827e-16,  1.95477878e-15,  2.13537656e-05],
        [ 4.85535702e-15, -4.41248513e-16, -2.07673277e-05]]),
 array([[-5.39803292e-16, -3.51334847e-16,  2.19504694e-05],
        [ 5.39803292e-16,  3.51334847e-16, -2.19504694e-05]]))

In [48]:
#W=S^-1F P is the way, cause of the orbital rotation
S=mf0.get_ovlp()
F=mf0.get_fock()
g_ijkl=mol0.intor('int2e_sph')
dF2el=np.einsum('ijkl,kl->ij',g_ijkl,dP)-0.5*np.einsum('ijkl,jl->ik',g_ijkl,dP)
dV=DeltaV(mol0,[.001,-.001])
dW_a3=np.linalg.inv(S)@(F+dV+dF2el)@(P+dP)-W0

## At the end the nuclear nuclear part 

In [49]:
"""def grad_nuc(mol, atmlst=None):
    gs = numpy.zeros((mol.natm,3))
    for j in range(mol.natm):
        q2 = mol.atom_charge(j)      <----------------------- derive here
        r2 = mol.atom_coord(j)
        for i in range(mol.natm):
            if i != j:
                q1 = mol.atom_charge(i)     <----------------------- and here 
                r1 = mol.atom_coord(i)      
                r = numpy.sqrt(numpy.dot(r1-r2,r1-r2))
                gs[j] -= q1 * q2 * (r2-r1) / r**3
    if atmlst is not None:
        gs = gs[atmlst]
    return gs
    """
pass

In [50]:
#is now easy to derive this function with respect to the nuclear charges
def alc_deriv_grad_nuc(mol,dL, atmlst=None):
    gs = np.zeros((mol.natm,3))
    for j in range(mol.natm):
        q2 =  mol.atom_charge(j) + dL[j]
        r2 = mol.atom_coord(j) 
        for i in range(mol.natm):
            if i != j:
                q1 = mol.atom_charge(i) +dL[i]
                r1 = mol.atom_coord(i)
                r = np.sqrt(np.dot(r1-r2,r1-r2))
                gs[j] -= q1 * q2 * (r2-r1) / r**3
    if atmlst is not None:
        gs = gs[atmlst]
    return gs

In [51]:
alc_deriv_grad_nuc(mol0,[.001,-.001])-g0.grad_nuc()

array([[ 0.        ,  0.        ,  0.00061698],
       [ 0.        ,  0.        , -0.00061698]])

In [52]:
g1.grad_nuc()-g0.grad_nuc()

array([[ 0.        ,  0.        ,  0.00061698],
       [ 0.        ,  0.        , -0.00061698]])

In [53]:
g0.grad_nuc()

array([[  0.        ,   0.        ,  14.81481481],
       [  0.        ,   0.        , -14.81481481]])